In [1]:
import credentials # Import from credentials.py
import settings # Import from settings.py 

import string
import nltk
from nltk.tokenize import RegexpTokenizer

import re
import tweepy
from tweepy import Stream
import mysql.connector
import pandas as pd
from textblob import TextBlob



In [3]:
class MyStreamListener(tweepy.Stream):

    def on_status(self, status):
        
        if status.retweeted:

            return True
        id_str = status.id_str
        created_at = status.created_at
        text = status.text    
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = status.user.location
        user_description = status.user.description
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        #This is because Twitter API has rate limits
 
        if status_code == 420:

            return False


mydb = mysql.connector.connect(
    host="localhost",
    user="Shamalan",
    passwd="abcd1234",
    database="shamalan",
    charset = 'utf8'
)
if mydb.is_connected():
    #To check if the table exists or to create a new one

    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

#myStreamListener = MyStreamListener()
myStream = MyStreamListener(credentials.API_KEY, credentials.API_SECRET_KEY, credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)

mydb.close()

North Pole is ending
However It could give a sorprise https://t.co/P8ECc8Jnrg
#ClimateChange #GlobalWarming… https://t.co/GKub8OKDdI
Long: None, Lati: None
RT @MichaelEMann: The extreme weather we're seeing in the Northern Hemisphere appears to be a consequence of "resonance". We've shown that…
Long: None, Lati: None
RT @DubaiCollege: Today the Arabian Ocean Rowing Team kick off DC Environment Awareness Week with a talk about their challenge to row acros…
Long: None, Lati: None
Frame 4 
Ya ne 🤔 black people are spectators in their own home
Long: None, Lati: None
RT @BotBicycle: Riding around town on your bike in Ohio, instead of operating a motor vehicle, will enormously decrease pollution, Climate…
Long: None, Lati: None
IMACASA 26" Machete, Black Poly Handle, Canvas Sheath Included, Full Tang - Optimal Balance and Force, 1075 High Ca… https://t.co/AlxxwE8dsu
Long: None, Lati: None
RT @mptblocks: 2/ after the @B1 saga, the devs that delivered $EOS decided they would instead work with 

KeyboardInterrupt: 